# Rules about Monotoinic Stacks
1. Monotonic Stacks always include the current index -- thus enumerate
2. Monotonic stacks have indices and not element(90% time)    -- sometimes elts are enough
3. Monotonic increasing/decreasing stack will have min/max so far at the first element
4. Should we use < or <=  -- depends -- 
   * Find Buildings               -- we need to pop equals            --> <=
   * Daily Temps, next greater    -- we need to wait for bigger temp  --> <

Extra Thinking
1. If you are unsure if a monotonic stack can work -- try monotonically inc/dec as input -- it will be clear


In [ ]:
# https://leetcode.com/problems/daily-temperatures/description/

from collections import deque
def dailyTemperatures(temperatures):
    res = [0] * len(temperatures)
    decreasing_stack = deque()
    for i, t in enumerate(temperatures):
        while decreasing_stack and temperatures[decreasing_stack[-1]] < t:
            idx = decreasing_stack.pop()
            res[idx] = i - idx
        decreasing_stack.append(i)
    return res

print(dailyTemperatures([73, 74, 75, 71, 69, 72, 76, 73])) # expected [1, 1, 4, 2, 1, 1, 0, 0]

[1, 1, 4, 2, 1, 1, 0, 0]


In [ ]:
# https://leetcode.com/problems/buildings-with-an-ocean-view/description/
# Easy Monotonic Stack
def findBuildings(heights):
    index_stack = []
    for ind, height in enumerate(heights):
        while index_stack and heights[index_stack[-1]] <= height:
            index_stack.pop()
        index_stack.append(ind)
    return index_stack


In [3]:
# https://leetcode.com/problems/next-greater-element-i/editorial/
# Elements based Mon Stack not indices
from collections import deque
def nextGreaterElement(nums1, nums2):
    d_stack, greater = deque(), {}
    for n in nums2:
        while d_stack and d_stack[-1] < n:
            greater[d_stack.pop()] = n
        d_stack.append(n)
    return [greater.get(n, -1) for n in nums1]

In [ ]:
# https://leetcode.com/problems/132-pattern/submissions/1207703452/
import math
# why use increasing stack and not a decreasing stack?
#  [1  3  2] -- 3 -> 2 tranistion is possble 

def find132pattern(nums):
    mon_dec_stack = [] #contains [num, min_before_num] 
    min_before_num = math.inf
    for num in nums:
        while mon_dec_stack and mon_dec_stack[-1][0]<=num: 
            mon_dec_stack.pop()
        if mon_dec_stack and mon_dec_stack[-1][1]< num: 
            return True
        mon_dec_stack.append([num, min_before_num])
        min_before_num = min(min_before_num, num)
    return False

In [ ]:
# https://leetcode.com/problems/remove-k-digits/description/
# we have to store the largest elt so far + min before 
# why a decreasing stack works and not an increasing stack?
# 
def removeKdigits(self, num: str, k: int) -> str:
    mon_inc_stack = []
    for digit in num:
        while k and mon_inc_stack and mon_inc_stack[-1] > digit:
            mon_inc_stack.pop()
            k -= 1
        mon_inc_stack.append(digit) 
    finalStack = mon_inc_stack[:-k] if k else mon_inc_stack 
    return "".join(finalStack).lstrip('0') or "0"

In [ ]:
# https://leetcode.com/problems/longest-continuous-subarray-with-absolute-diff-less-than-or-equal-to-limit/
# mon_decreasing_stack has max of the window at its 0th element
# mon_increasing_stack has min of the window at its 0th element
# These can be updated as we slide the left pointer by simply poping

from collections import deque

def longestSubarray(nums, limit) -> int:
        dec_stack = deque()  # decreasing deque for max
        inc_stack = deque()  # increasing deque for min
        l = 0
        res = 0
        for r, num in enumerate(nums):
            while dec_stack and num > dec_stack[-1]:
                dec_stack.pop()
            dec_stack.append(num)
            while inc_stack and num < inc_stack[-1]:
                inc_stack.pop()
            inc_stack.append(num)
            # as long as we have max-min > limit -- we are in violation
            while dec_stack[0] - inc_stack[0] > limit: 
                if dec_stack[0] == nums[l]: #  if l is current max advance l, update stack
                    dec_stack.popleft()
                if inc_stack[0] == nums[l]: #  if l is current min advance l, update stack
                    inc_stack.popleft()
                l += 1
            res = max(res, r - l + 1)
        return res

import heapq

def longestSubarray2(nums, limit) -> int:
        ans = 1
        min_hq = [] 
        max_hq = [] 
        l = 0
        for r in range(len(nums)):
            heapq.heappush(min_hq, (nums[r],r))
            heapq.heappush(max_hq, (-nums[r],r))

            while min_hq[0][1] < l:
                heapq.heappop(min_hq)
            while max_hq[0][1] < l:
                heapq.heappop(max_hq)

            if -max_hq[0][0] - min_hq[0][0] <= limit:
                ans = max(ans, r - l + 1)
            else:
                l += 1
        return ans

In [ ]:
# https://leetcode.com/problems/sum-of-subarray-minimums/editorial/

# Monotonic Stack ulti Problem

def sumSubarrayMins(arr):
        mon_stack = []
        total = 0
        for r in range(len(arr)+1):
            while mon_stack and (r==len(arr) or arr[r] < arr[mon_stack[-1]]): 
                pop_ind = mon_stack.pop()
#               |----subarrays where arr[pop_ind]=min----|
#               Start ↓         pop_ind            End ↓
#               |----------------|------------------|
#               l+1           ←      →             r-1
#                [l + 1, pop_ind]  x [pop_ind, r - 1]
                l = mon_stack[-1] if mon_stack else -1
                total +=  (r-pop_ind) * (pop_ind - l) * arr[pop_ind]
            mon_stack.append(r)
        return total%(10**9 + 7)